In [1]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

/Users/akanodia/opt/anaconda3/envs/rag_app/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
def load():
    #Load the PDF Document
    loader = PyPDFLoader("documents/AI in 2024.pdf")
    pages = loader.load_and_split()
    '''
    loader.load will load the entire pdf in 1 document object.
    load_and_split create separate document object for every split.
    '''
    #split pages by char
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 100,
        length_function = len,
        add_start_index = True
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")
    return chunks

In [57]:
document_chunks = load()

Split 15 documents into 38 chunks.


In [2]:
#Initialise the models 

#ollama pull qwen3-embeddings
embeddings = OllamaEmbeddings(model="qwen3-embedding:0.6b")

#Initialise vector store
vectordb = Chroma(collection_name="documents", embedding_function = embeddings, persist_directory="./db/chroma_db") #path to save data locally

In [4]:
vectordb.delete_collection()

In [59]:
#store documents in vectordb
document_ids = vectordb.add_documents(documents=document_chunks)
print(document_ids[:3])


['ab44886a-9128-42d0-91d0-06aee60d07e9', '752cec16-69c1-4ce9-9298-7132a2b4abc9', '9fc4854f-f14d-454b-8b3f-557338c4d76e']


In [60]:
#Define the retrieval chain
retriever = vectordb.as_retriever(search_type = "similarity",
                                  search_kwargs={
                                        "k":4
                                  })

In [61]:
#ollama pull qwen3
llm = ChatOllama(model="qwen3:0.6b",keep_alive="2h", temperature=0)

In [62]:
#Define the chat prompt
chat_prompt = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant. Use the context to answer the question. If you don’t know, say you don’t know."),
        ("human", "{context}\n\nQuestion: {question}")
    ]
)


In [63]:
#Retrieve context

retrieve_context = RunnableLambda(
    lambda input_dict: {
        "context": "\n\n".join(
            doc.page_content for doc in retriever.invoke(input_dict["question"])
        ),
        "question": input_dict["question"],
    }
)

In [64]:
pipeline = retrieve_context | chat_prompt | llm

In [65]:
#Execute the pipeline
input_data = {
    "question": "What are the key trends in AI for 2024?"}
result = pipeline.invoke(input_data)
print(result)

content="The key trends in AI for 2024, as highlighted in the context, include topics such as **Dataiku's advancements in AI integration**, **Deloitte's focus on business transformation through AI**, and **Snowflake's role in data analytics**. These trends are part of the 2024 hot takes from the specified companies, which are already covered in the provided information." additional_kwargs={} response_metadata={'model': 'qwen3:0.6b', 'created_at': '2025-10-28T14:22:16.891843Z', 'done': True, 'done_reason': 'stop', 'total_duration': 10785232083, 'load_duration': 6423877375, 'prompt_eval_count': 227, 'prompt_eval_duration': 535375041, 'eval_count': 455, 'eval_duration': 3649295927, 'model_name': 'qwen3:0.6b', 'model_provider': 'ollama'} id='lc_run--c0de88e2-b5df-490c-ad4d-a9de87f57a0a-0' usage_metadata={'input_tokens': 227, 'output_tokens': 455, 'total_tokens': 682}


In [67]:
# add output parser to pipeline to extract answer only
output_parser = StrOutputParser()

In [68]:
pipeline_with_parser = pipeline | output_parser

In [70]:
#Execute the pipeline with output parser
input_data = {
    "question": "What challenges exist regarding AI adoption?"}
result = pipeline_with_parser.invoke(input_data)
print(result)

The challenges related to AI adoption include:  
1. **Accepting AI's limitations** to avoid unintended consequences and plan responsibly.  
2. **Balancing innovation with forward-thinking** to manage control mechanisms and technological guardrails.  
3. **Addressing trust erosion** by redefining accountability and responsibilities across the organization.  
4. **Ensuring transparent and open communication** with stakeholders, beyond just regulatory compliance, to maintain trust and collaboration.
